<a href="https://colab.research.google.com/github/ethanosaurus6/Proyek-Akhir-Jaringan-Saraf-Tiruan/blob/main/Pengujian_Cross_Validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Stress-Lysis.csv to Stress-Lysis (15).csv


In [ ]:
df = pd.read_csv('Stress-Lysis.csv')
df.head()

,Humidity,Temperature,Step count,Stress Level
0,21.33,90.33,123,1
1,21.41,90.41,93,1
2,27.12,96.12,196,2
3,27.64,96.64,177,2
4,10.87,79.87,87,0


In [ ]:
import numpy as np

class LVQ:
    def __init__(self):
        # Inisialisasi atribut kelas LVQ
        # weights: array kosong untuk menyimpan bobot prototipe (representasi setiap kelas)
        # labels: array kosong untuk menyimpan label yang sesuai dengan bobot
        self.weights = np.array([])
        self.labels = np.array([])

    def train(self, training_data, target_labels, learning_rate, decay_factor, max_epochs):
        # Mendapatkan label unik dan indeks pertama dari masing-masing label dalam data latih
        unique_labels, initial_indices = np.unique(target_labels, return_index=True)

        # Menyimpan label unik sebagai representasi kelas
        self.labels = unique_labels

        # Inisialisasi bobot awal dengan data latih yang mewakili setiap label unik
        self.weights = training_data[initial_indices].astype(np.float64)

        # Menghapus data latih yang sudah dipakai sebagai bobot awal dari data latih
        training_data = np.delete(training_data, initial_indices, axis=0)
        target_labels = np.delete(target_labels, initial_indices, axis=0)

        epoch = 0  # Mengatur epoch awal
        while epoch < max_epochs:  # Looping hingga mencapai jumlah maksimum epoch
            for i, sample in enumerate(training_data):
                # Menghitung jarak Euclidean kuadrat antara sampel dan setiap bobot
                distances = [np.sum((weight - sample) ** 2) for weight in self.weights]

                # Menentukan indeks bobot prototipe terdekat
                closest_index = np.argmin(distances)

                # Menentukan tanda penyesuaian berdasarkan apakah label cocok
                adjustment_sign = 1 if target_labels[i] == self.labels[closest_index] else -1

                # Menyesuaikan bobot prototipe terdekat (mendekatkan atau menjauhkan)
                self.weights[closest_index] += adjustment_sign * learning_rate * (sample - self.weights[closest_index])

            # Mengurangi learning rate dengan faktor peluruhan
            learning_rate *= decay_factor

            # Menambah jumlah epoch
            epoch += 1

    def predict(self, input_data):
        # Inisialisasi daftar untuk menyimpan prediksi
        predictions = []

        for sample in input_data:  # Iterasi melalui setiap sampel dalam data masukan
            # Menghitung jarak Euclidean kuadrat antara sampel dan setiap bobot
            distances = [np.sum((weight - sample) ** 2) for weight in self.weights]

            # Menentukan label dari bobot prototipe terdekat dan menambahkannya ke prediksi
            predictions.append(self.labels[np.argmin(distances)])

        # Mengembalikan array prediksi
        return np.array(predictions)

In [ ]:
def calculate_accuracy(predicted_labels, true_labels):
    # Membandingkan setiap label prediksi dengan label sebenarnya
    # Jika cocok, tambahkan 1 ke daftar correct_predictions, jika tidak tambahkan 0
    correct_predictions = [1 if predicted_labels[i] == true_labels[i] else 0 for i in range(len(predicted_labels))]

    # Mengembalikan akurasi dengan membagi jumlah prediksi benar dengan total jumlah prediksi
    return sum(correct_predictions) / len(predicted_labels)

In [ ]:
# Pranormalisasi data
scaler = MinMaxScaler()
features = df[['Humidity', 'Temperature', 'Step count']]
scaled_features = scaler.fit_transform(features)

# Encoding target label
encoder = LabelEncoder()
target = encoder.fit_transform(df['Stress Level'])

# # Membagi dataset menjadi data latih dan data uji , random_state=42
# X_train, X_test, y_train, y_test = train_test_split(scaled_features, target, test_size=0.2)

In [ ]:
# Split dataset into 3 parts
num_samples = len(df)
part_size = num_samples // 3
indices = np.arange(num_samples)
np.random.shuffle(indices)

part1_indices = indices[:part_size]
part2_indices = indices[part_size:2 * part_size]
part3_indices = indices[2 * part_size:]

parts = [part1_indices, part2_indices, part3_indices]

In [ ]:
# Initialize variables
learning_rate = 0.3
decay_factor = 0.6
max_epochs = 100

print("No\tData Latih\tData Uji\tAkurasi")

for i, test_indices in enumerate(parts):
    train_indices = np.hstack([parts[j] for j in range(3) if j != i])

    X_train, y_train = scaled_features[train_indices], target[train_indices]
    X_test, y_test = scaled_features[test_indices], target[test_indices]

    lvq = LVQ()
    lvq.train(X_train, y_train, learning_rate, decay_factor, max_epochs)
    predictions = lvq.predict(X_test)
    accuracy = calculate_accuracy(predictions, y_test)

    train_parts = ["K" + str(j + 1) for j in range(3) if j != i]
    test_part = "K" + str(i + 1)

    print(f"{i + 1}\t{', '.join(train_parts)}\t{test_part}\t{accuracy}")


No	Data Latih	Data Uji	Akurasi
1	K2, K3	K1	0.9490254872563718
2	K1, K3	K2	0.9445277361319341
3	K1, K2	K3	0.9490254872563718
